In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [5]:
#Innisfail coordinates

latmin = -17.99 
latmax = -16 
lonmin = 147.0  
lonmax = 148.99  

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')


In [6]:
boxLat = []
boxLon = []
boxWh = []
boxT = []

for u in range(len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]
        t = ncs.variables['TIME'][:]

        # Check if satellite points are in the box
        id1 = lats>latmin 
        id2 = lats<latmax
        id3 = lons>lonmin
        id4 = lons<lonmax
        id1 = id1*1
        id2 = id2*1
        id3 = id3*1
        id4 = id4*1
        tot = id1+id2+id3+id4
        # If this is the case then the sum should be 4
        inside = np.where(tot==4)[0]

        if len(inside)>0:
            boxLat.append(lats[inside])
            boxLon.append(lons[inside])
            boxWh.append(wh[inside])
            boxT.append(t[inside])
        

In [8]:
len(boxLat)

38

In [9]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        t = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        t = np.append(t,boxT[k])

In [22]:
print lat.shape
print lon.shape
print wh.shape
print t.shape

(127376,)
(127376,)
(127376,)
(127376,)


In [11]:
# ncs = NetCDFFile(jason2URL[0])
# start = dt.datetime(1950,1,1)
# # Get desired time step  
# time_var = ncs.variables['TIME']
# itime = netCDF4.date2index(start,time_var,select='nearest')
# dtime = netCDF4.num2date(time_var[itime],time_var.units)
# daystr = dtime.strftime('%Y-%b-%d %H:%M:%S')
# print 'Saral record start time:',daystr

# end = dt.datetime(2020,2,1)
# # Get desired time step  
# time_var = ncs.variables['TIME']
# itime2 = netCDF4.date2index(end,time_var,select='nearest')
# dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
# dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
# #h=nc.variables['SWH_KU_CAL']
# print 'Saral record end time:',dayend

Saral record start time: 2008-Jul-07 22:22:57
Saral record end time: 2018-Jul-13 08:05:15


In [12]:
print min(wh)

-32768.0


In [13]:
ids = np.where(wh>0)[0]
print ids

[     0      1      2 ..., 127373 127374 127375]


In [25]:
ids = np.where(wh>0)[0]
print ids
whpos = wh[ids]
print len(whpos)

#wave height positive values

[     0      1      2 ..., 127373 127374 127375]
126906


In [24]:
print np.mean(whpos)
print max(whpos)
print min (whpos)
print  min(lon)
print max(lon)
print  min(lat)
print max(lat)

1.4437237978
7.409
0.001
147.0
148.99
-17.99
-16.0001


In [23]:
print  min(lon)
print max(lon)

print  min(lat)
print max(lat)

print np.mean(lat)
print np.mean(lon)

147.0
148.99
-17.99
-16.0001
-17.0024
147.8
